# Convolutional neural network tested on the mnist data


https://martin-thoma.com/tensor-flow-quick/

In [40]:
import tensorflow as tf
import numpy as np

In [2]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/data/", one_hot=True)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


#### Dataset Mnist

In [3]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [4]:
mnist.train.images.shape

(55000, 784)

In [5]:
mnist.train.labels.shape

(55000, 10)

In [6]:
mnist.train.num_examples

55000

## Define the model

In [7]:
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 20

In [8]:
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

In [9]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [10]:
# Create model
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], 
                                                  padding='SAME'),b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, _dropout)

    # Convolution Layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    conv2 = tf.nn.dropout(conv2, _dropout)

    # Fully connected layer
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout) # Apply Dropout

    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

In [11]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), 
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])), 
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])), 
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [12]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

In [13]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [33]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#### Train the model and save it to file

Notice that we will use  ```saver = tf.train.Saver()``` to save the model into disk

    saver = tf.train.Saver()
    # Stuff here: train the model
    saver.save(sess, './model')

Doing so we will save the session into ```./mode```.

Afterward we ran restore the  session 




In [15]:
# Launch the graph
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    saver = tf.train.Saver()    

    step = 1
    
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print ("Iter " , str(step*batch_size) , ", Minibatch Loss= " , 
                  "{:.6f}".format(loss) , ", Training Accuracy= " , "{:.5f}".format(acc))
        
        step += 1
        
    print ("Optimization Finished!")
    
    print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images, 
                                                              y: mnist.test.labels, 
                                                              keep_prob: 1.}))
    saver.save(sess, './model')


Iter  2560 , Minibatch Loss=  12293.529297 , Training Accuracy=  0.35156
Iter  5120 , Minibatch Loss=  7141.931641 , Training Accuracy=  0.60938
Iter  7680 , Minibatch Loss=  8345.753906 , Training Accuracy=  0.58594
Iter  10240 , Minibatch Loss=  4800.775391 , Training Accuracy=  0.68750
Iter  12800 , Minibatch Loss=  4279.565430 , Training Accuracy=  0.72656
Iter  15360 , Minibatch Loss=  2340.699707 , Training Accuracy=  0.75000
Iter  17920 , Minibatch Loss=  1896.912354 , Training Accuracy=  0.82812
Iter  20480 , Minibatch Loss=  1378.325317 , Training Accuracy=  0.87500
Iter  23040 , Minibatch Loss=  1783.575439 , Training Accuracy=  0.86719
Iter  25600 , Minibatch Loss=  2055.001465 , Training Accuracy=  0.82031
Iter  28160 , Minibatch Loss=  1484.470093 , Training Accuracy=  0.87500
Iter  30720 , Minibatch Loss=  1719.613403 , Training Accuracy=  0.85938
Iter  33280 , Minibatch Loss=  1617.190918 , Training Accuracy=  0.88281
Iter  35840 , Minibatch Loss=  685.861206 , Training 

### Reload the moden inside a with statement using a new session 

In [23]:
with tf.Session() as sess2:
    loader = tf.train.import_meta_graph('model.meta')
    loader.restore(sess2, tf.train.latest_checkpoint('./'))
    batch_xs, batch_ys = mnist.test.next_batch(10000)
    print ("Prediction Accuracy:", sess2.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.}))


Prediction Accuracy: 0.9263


#### Reload the model and use it

Once our model has been trained, how can we use it in order to predict the class for a new image?

In [35]:
loader = tf.train.import_meta_graph('model.meta')

In [36]:
sess2 = tf.Session()

In [37]:
loader.restore(sess2, tf.train.latest_checkpoint('./'))

In [38]:
#output of the network for a given input

sess2.run(pred, feed_dict={x: mnist.train.images[0:1], y: batch_ys, keep_prob: 1.})

array([[ -1864.12023926,  18024.63867188,  -7051.08300781,  -4062.22607422,
          7181.37890625,  -2401.61328125,   1641.59973145,   3584.84716797,
          -611.53094482,   -857.90240479]], dtype=float32)

In [41]:
np.argmax(sess2.run(pred, feed_dict={x: mnist.train.images[0:1], y: batch_ys, keep_prob: 1.}))

1